In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
import pandas as pd
from xgboost import XGBRegressor
import category_encoders as ce
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from pdpbox.pdp import pdp_interact, pdp_interact_plot, pdp_isolate, pdp_plot
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV, cross_validate
from sklearn.impute import SimpleImputer
import sklearn.metrics as metrics
from xgboost import XGBClassifier, plot_importance
import uuid 


In [ ]:
a = uuid.uuid1()

In [ ]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
pcs = pd.read_csv('data/pcs.csv')

In [ ]:
pcs.head()

In [ ]:
pcs.drop("survey_year", axis=1, inplace=True)

In [ ]:
pcs.shape

Lets check out how many people on this sruvey has mental illness

In [ ]:
pcs['mental_illness'].value_counts(normalize=True)

In [ ]:
pcs['mental_illness'].hist()

In [ ]:
sns.countplot(x="living_situation", palette="ch:.25", data=pcs, )

The overwheming majority huh......

In [ ]:
pcs['age_group'].value_counts(normalize=True)

Children account for approximately 18 percent of mental illness patients

In [ ]:
pcs['sex'].value_counts(normalize=True)

In [ ]:
pcs['race'].value_counts(normalize=True)

In [ ]:
pcs['hispanic_ethnicity'].value_counts(normalize=True)

In [ ]:
pcs['serious_mental_illness'].value_counts(normalize=True)

Strangely enough, 84.3 percent of respondants have a serious mental illness, along with 94.9 percent who havve a mental illness. * note this is a data leakage issue in future modeling 

In [ ]:
f, axes = plt.subplots( sharey=True, figsize=(11, 4))
sns.countplot(x="living_situation", palette="ch:.25", data=pcs, )

In [ ]:
pcs['living_situation'].value_counts(normalize=True)

Most live in a private residence, but the second largest feature is other. With little info on that, I become curious as to the true state of 20% of the respondants

In [ ]:
f, axes = plt.subplots( sharey=True, figsize=(11, 4))
sns.countplot(y="household_composition", palette="ch:1.3,-.2,dark=.3", data=pcs, )
plt.xlabel("Count")
plt.ylabel("Living Status")

In [ ]:
f, axes = plt.subplots( sharey=True, figsize=(11, 4))
plt.xticks(rotation='vertical')
sns.countplot(y="employment_status", palette="ch:2.5,-.2,dark=.3", data=pcs)
plt.xlabel("Count")
plt.ylabel("Employment Status")

In [ ]:
pcs['three_digit_residence_zip_code'].value_counts(normalize=True)

In [ ]:
 l = pcs['three_digit_residence_zip_code'].value_counts(normalize=True)

In [ ]:
l.head(2).sum()

Almost 25% of the group are accounted for with two zipcodes. With a wikipedia search, we see that comprises Brooklyn and The Bronx.

In [ ]:
#importing plotly and cufflinks in offline mode
import cufflinks as cf
import plotly.offline
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

In [ ]:
pcs.serious_mental_illness.value_counts().sort_values(ascending=False).iplot(kind='bar', yTitle='Number of Complaints', 
title='Number complaints in each product')

In [ ]:
pcs['id'] = [ uuid.uuid1()  for k in pcs.index]

In [ ]:
target = 'serious_mental_illness'
X = pd.DataFrame(pcs.drop(columns=[target,'mental_illness']))
y = pd.DataFrame(pcs[target])

In [ ]:
X.shape

In [ ]:
X_trainval, X_test, y_trainval, y_test = train_test_split(
    X, 
    y, 
    test_size=0.001,
    train_size=0.10, 
    stratify=y, 
    random_state=42
)

X_train, X_val, y_train, y_val = train_test_split(
    X_trainval, 
    y_trainval, 
    test_size=0.05,
    train_size=0.10, 
    stratify=y_trainval, 
    random_state=42
)

print('X_train shape', X_train.shape)
print('y_train shape', y_train.shape)
print('X_val shape', X_val.shape)
print('y_val shape', y_val.shape)
print('X_test shape', X_test.shape)
print('y_test shape', y_test.shape)

In [ ]:
train_id = X_train['id']
val_id = X_val['id']
test_id = X_test['id']

In [53]:
x_processor = make_pipeline(
    ce.OrdinalEncoder(), 
    SimpleImputer(strategy='median')
)

y_processor = make_pipeline(
    ce.OrdinalEncoder(), 
    SimpleImputer(strategy='median')
)

In [ ]:
cols = X.columns
len(cols)

In [54]:
def prepare_inputs(X_train, X_val, X_test):
	X_train_enc = pd.DataFrame(x_processor.fit_transform(X_train), columns=cols)
	X_val_enc = pd.DataFrame(x_processor.transform(X_val), columns=cols)
	X_test_enc = pd.DataFrame(x_processor.transform(X_test), columns=cols)
	return X_train_enc, X_val_enc, X_test_enc

def prepare_targets(y_train, y_val, y_test):
    y_train_enc = pd.DataFrame(y_processor.fit_transform(y_train), columns=['serious_mental_illness'])
    y_val_enc = pd.DataFrame(y_processor.transform(y_val), columns=['serious_mental_illness'])
    y_test_enc = pd.DataFrame(y_processor.transform(y_test), columns=['serious_mental_illness'])
    return y_train_enc, y_val_enc, y_test_enc

In [ ]:
X_train_processed, X_val_processed, X_test = prepare_inputs(X_train, X_val, X_test)

In [52]:
X_train_processed.head()

,program_category,region_served,age_group,sex,transgender,sexual_orientation,hispanic_ethnicity,race,living_situation,household_composition,preferred_language,veteran_status,employment_status,number_of_hours_worked_each_week,education_status,special_education_services,intellectual_disability,autism_spectrum,other_developmental_disability,alcohol_related_disorder,drug_substance_disorder,mobility_impairment_disorder,hearing_visual_impairment,hyperlipidemia,high_blood_pressure,diabetes,obesity,heart_attack,stroke,other_cardiac,pulmonary_asthma,alzheimer_or_dementia,kidney_disease,liver_disease,endocrine_condition,neurological_condition,traumatic_brain_injury,joint_disease,cancer,no_chronic_med_condition,unknown_chronic_med_condition,smokes,received_smoking_medication,received_smoking_counseling,principal_diagnosis_class,additional_diagnosis_class,ssi_cash_assistance,ssdi_cash_assistance,veterans_disability_benefits,veterans_cash_assistance,public_assistance_cash_program,other_cash_benefits,medicaid_and_medicare_insurance,no_insurance,unknown_insurance_coverage,medicaid_insurance,medicaid_managed_insurance,medicare_insurance,private_insurance,child_health_plus_insurance,family_health_plus_insurance,other_insurance,criminal_justice_status,three_digit_residence_zip_code,other_chronic_med_condition,id
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,999.0,1.0,1.0
1,1.0,1.0,2.0,1.0,1.0,2.0,2.0,1.0,2.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,2.0,2.0,1.0,2.0,1.0,1.0,1.0,2.0,148.0,1.0,2.0
2,1.0,2.0,1.0,2.0,1.0,2.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,3.0,2.0,2.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,117.0,2.0,3.0
3,1.0,3.0,2.0,1.0,1.0,2.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,3.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,3.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,1.0,1.0,3.0,112.0,2.0,4.0
4,1.0,4.0,1.0,1.0,1.0,2.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,3.0,2.0,2.0,2.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,2.0,1.0,1.0,1.0,3.0,107.0,2.0,5.0


In [55]:
y_train_processed, y_val_processed, y_test_processed = prepare_targets(y_train, y_val, y_test)

In [56]:
y_train_processed.head()

,serious_mental_illness
0,1.0
1,1.0
2,2.0
3,2.0
4,2.0


In [ ]:
eval_set = [(X_train_processed, y_train_processed), (X_val_processed, y_val_processed)]

In [57]:
xgbcl = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1.0,
         gamma=0.0, max_delta_step=0.0, min_child_weight=1.0,
         missing=None, n_jobs=-1, objective='binary:logistic', random_state=42, reg_alpha=0.0,
         reg_lambda=1.0, scale_pos_weight=1.0, tree_method='auto')

kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

param_grid = { 
    'colsample_bytree':[.75,1],
    'learning_rate':[0.01,0.05,0.1,0.3,0.5],
    'max_depth':[1,2,3,5],
    'subsample':[.75,1],
    'n_estimators': list(range(50, 400, 50))
}

In [ ]:
grid_search = GridSearchCV(estimator=xgbcl, param_grid=param_grid, n_jobs=-1, cv=kfold)
grid_result = grid_search.fit(X_train_processed, y_train_processed)

print(f'Best: {grid_result.best_score_} using {grid_result.best_params_}','\n')

In [ ]:
xgbcl = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1.0,
         gamma=0.0, max_delta_step=0.0, min_child_weight=1.0,
         missing=None, n_jobs=-1, objective='multi:softmax', num_class=3,random_state=42, reg_alpha=0.0,
         reg_lambda=1.0, scale_pos_weight=1.0, tree_method='auto',
         colsample_bytree = grid_result.best_params_['colsample_bytree'], 
         learning_rate = grid_result.best_params_['learning_rate'], 
         max_depth = grid_result.best_params_['max_depth'], 
         subsample = grid_result.best_params_['subsample'], 
         n_estimators = grid_result.best_params_['n_estimators'])

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

#refit the model on k-folds to get stable avg error metrics
scores = cross_validate(estimator=xgbcl, X=X_train_processed, y=y_train_processed, cv=kfold, scoring=['accuracy', ], n_jobs=-1)

In [ ]:
print('Training 5-fold Cross Validation Results:\n')
# print('AUC: ', scores['test_roc_auc'].mean())
print('Accuracy: ', scores['test_accuracy'].mean())
# print('Precision: ', scores['test_precision'].mean())
# print('Recall: ', scores['test_recall'].mean())
# print('F1: ', scores['test_f1'].mean(), '\n')

In [ ]:
from sklearn import metrics

xgbcl.fit(X_train_processed, y_train_processed)

#Generate predictions against our training and test data
pred_train = xgbcl.predict(X_train_processed)
proba_train = xgbcl.predict_proba(X_train_processed)
pred_test = xgbcl.predict(X_val_processed)
proba_test = xgbcl.predict_proba(X_val_processed)

# Print model report
print("Classification report (Test): \n")
print(metrics.classification_report(y_val_processed, pred_test))

print ('\nTrain Accuracy:', metrics.accuracy_score(y_train_processed, pred_train))
print ('Test Accuracy:', metrics.accuracy_score(y_val_processed, pred_test))



In [ ]:
plot_importance(xgbcl, importance_type='gain')

In [ ]:
from pdpbox.pdp import pdp_interact, pdp_interact_plot

In [ ]:
features = ['race', 'cancer']

interaction = pdp_interact(
    model=xgbcl,
    dataset=X_val_processed,
    model_features=X_val_processed.columns,
    features=features
)

In [ ]:
pdp_interact_plot(interaction, plot_type='grid',
                  feature_names=features)

In [ ]:

# model = XGBClassifier(    
#     n_estimators=1000,  
#     max_depth=7,       
#     learning_rate=0.5,  
#     n_jobs=-1
#     )
# model.fit(X_train_processed, y_train_processed, eval_set=eval_set, early_stopping_rounds=10)